In [60]:
import pandas as pd
import numpy as np

from google.colab import drive

# Bibliotecas de pré-processamento
from sklearn.preprocessing import LabelEncoder

# Bibliotecas de plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, PassiveAggressiveClassifier, SGDClassifier
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [61]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [62]:
df = pd.read_csv('/content/drive/MyDrive/Data/merged_df.csv')

In [63]:
from sklearn.preprocessing import StandardScaler

df = df.drop(['record_id', 'idade_faixa'], axis=1)

X_afetam_eficacia = df[['idade_diagnostico','primeiro_IMC','ultima_informacao_paciente', 'tempo_seguimento', 'recidiva_distancia', 'recidiva_regional', 'recidiva_local']]
X_afetam_eficacia[['tempo_seguimento']] = StandardScaler().fit_transform(X_afetam_eficacia[['tempo_seguimento']])

In [64]:
km = KMeans( n_clusters = 2, init = 'random', max_iter = 300, n_init = 100, random_state = 52 )
km.fit( X_afetam_eficacia )
y_km = km.predict( X_afetam_eficacia )
df['eficacia_tratamento'] = y_km

In [65]:
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 1")))

# Este é um sistema de classificação de tratamento em que "0" significa que o tratamento foi bom e "1" significa 
# que foi ruim. Há dois casos apresentados: no primeiro, uma pessoa idosa de 80 anos sobreviveu ao tratamento, indicando 
# que ele foi bom. No segundo caso, não houve sobreviventes entre as pessoas mais velhas e, portanto, o tratamento foi classificado 
# como negativo para essa faixa etária. É importante lembrar que a eficácia do tratamento pode variar de acordo com vários fatores e 
# que cada caso é único.

0
38


In [66]:
df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")

print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")))

# Este modelo classifica o tratamento como bom (0) ou ruim (1). Ele foi testado em dois casos de mulheres com menos
# de 30 anos: uma morreu e a outra sobreviveu. O modelo foi capaz de classificar corretamente cada caso, indicando que 
# tem uma boa capacidade de classificação.

12
0


In [67]:
df = df.query("tratamento == 0 or tratamento == 1")[:]

In [68]:
import numpy as np

df = df.astype('float64')
df = df[np.isfinite(df).all(1)]

In [69]:
from sklearn.model_selection import train_test_split

X = df.drop(['tratamento'], axis=1)
y = df['tratamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 73)

In [70]:
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:07<00:00,  3.86it/s]


In [72]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.67,0.67,0.67,0.67,0.33
RandomForestClassifier,0.67,0.67,0.67,0.67,0.60
LGBMClassifier,0.67,0.66,0.66,0.66,0.22
ExtraTreesClassifier,0.66,0.66,0.66,0.66,0.50
LinearSVC,0.66,0.66,0.66,0.66,0.36
RidgeClassifierCV,0.66,0.66,0.66,0.66,0.06
LogisticRegression,0.66,0.66,0.66,0.66,0.04
SVC,0.66,0.66,0.66,0.66,0.37
LinearDiscriminantAnalysis,0.66,0.65,0.65,0.66,0.11


In [73]:
classifiers = [AdaBoostClassifier(), 
               RandomForestClassifier(),
               ExtraTreesClassifier(),
               RidgeClassifierCV(), 
               LinearSVC(),
               LogisticRegression(), 
               SVC(), 
               LGBMClassifier(),
               LinearDiscriminantAnalysis(), 
               RidgeClassifier(), 
               CalibratedClassifierCV(), 
               XGBClassifier(),
               NearestCentroid(), 
               SGDClassifier(), 
               BaggingClassifier(),
               BernoulliNB(),
               PassiveAggressiveClassifier(), 
               Perceptron(),
               NuSVC(), 
               DecisionTreeClassifier(), 
               KNeighborsClassifier(),
               LabelSpreading(),
               LabelPropagation(), 
               ExtraTreeClassifier(),
               GaussianNB(),
               DummyClassifier(),
               QuadraticDiscriminantAnalysis()]

precisions = []
recalls = []

for classifier in classifiers:
    clf = classifier.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precisions.append(precision_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))

In [74]:
models['Precision'] = 0
models['Recall'] = 0   

for i in range(0, len(precisions)):
  models['Precision'].iloc[i] = precisions[i]
  models['Recall'].iloc[i] = recalls[i]

models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Precision,Recall
Model,,,,,,,
AdaBoostClassifier,0.67,0.67,0.67,0.67,0.33,0.66,0.62
RandomForestClassifier,0.67,0.67,0.67,0.67,0.60,0.67,0.59
LGBMClassifier,0.67,0.66,0.66,0.66,0.22,0.66,0.61
ExtraTreesClassifier,0.66,0.66,0.66,0.66,0.50,0.65,0.60
LinearSVC,0.66,0.66,0.66,0.66,0.36,0.47,1.00
RidgeClassifierCV,0.66,0.66,0.66,0.66,0.06,0.63,0.60
LogisticRegression,0.66,0.66,0.66,0.66,0.04,0.00,0.00
SVC,0.66,0.66,0.66,0.66,0.37,0.65,0.60
LinearDiscriminantAnalysis,0.66,0.65,0.65,0.66,0.11,0.64,0.59


In [75]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6716899892357373

##GridSearch

In [ ]:
model = AdaBoostClassifier()

params = {
          'estimator':[None, DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2)],
          'n_estimators': [50, 150, 100, 200, 300, 400, 500],
          'learning_rate': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],
          'algorithm': ['SAMME', 'SAMME.R']
          }

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier()

params = {
          'n_estimators': [50, 150, 100, 200, 300, 400, 500],
          'criterion': ['gini','entropy','log_loss'],
          'max_depth': [5,10,15,20,25],
          'min_samples_split':[1,2,3,4],
          'min_samples_leaf': [1,2,3,4],
          'min_weight_fraction_leaf':[0,1,2],
          'max_features':['sqrt','log2',None],
          'max_leaf_nodes': [None, 1, 2],
          'min_impurity_decrease': [0, 1, 2],
          'bootstrap': [True, False],
          'oob_score': [True, False],
          'n_jobs': [None, 1, -1],
          'verbose': [0, 1, 2],
          'warm_start': [True, False], 
          'class_weight': ['balanced', 'balanced_subsample'],
          'ccp_alpha': [0, 1]
          }

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.score(X_test, y_test))

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()

params = {
    'boosting_type': ['gbdt', 'dart', 'goss'],
    'num_leaves': [20, 30, 40],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150],
    'min_child_samples': [5, 10, 15],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'reg_alpha': [0, 0.1, 1.0],
    'reg_lambda': [0, 0.1, 1.0],
    'importance_type': ['split', 'gain']
}

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.score(X_test, y_test))

##RandomizedSearchCV

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

model = AdaBoostClassifier()

params = {
          'estimator':[None, DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2)],
          'n_estimators': [50, 150, 100, 200, 300, 400, 500],
          'learning_rate': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],
          'algorithm': ['SAMME', 'SAMME.R']
          }

model_random = RandomizedSearchCV(estimator=model, param_distributions=params, cv=5, n_iter=50)

model_random.fit(X_train, y_train)

print(model_random.best_params_)
print(model_random.best_score_)
print(model_random.score(X_test, y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

random_grid = {
            'n_estimators': [50, 150, 100, 200, 300, 400, 500],
            'max_depth': [5,10,15,20,25,None],
            'min_samples_split':[2,3,4,5],
            'min_samples_leaf': [1,2,3,4],
            'min_weight_fraction_leaf':[0,1,2],
            'max_features':['sqrt','log2',None],
            'max_leaf_nodes': [None, 1, 2],
            'min_impurity_decrease': [0, 1, 2],
            'bootstrap': [True, False],
            'oob_score': [True, False],
            'n_jobs': [None, 1, -1],
            'verbose': [0, 1, 2],
            'warm_start': [True, False],
            'ccp_alpha': [0, 1]
               }

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)

print(rf_random.best_params_)
print(rf_random.best_score_)
print(rf_random.score(X_test, y_test))

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()

params = {
    'boosting_type': ['gbdt', 'dart', 'goss'],
    'num_leaves': [20, 30, 40],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150],
    'min_child_samples': [5, 10, 15],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'reg_alpha': [0, 0.1, 1.0],
    'reg_lambda': [0, 0.1, 1.0],
    'importance_type': ['split', 'gain']
}

grid_search = RandomizedSearchCV(estimator=model, param_distributions=params, cv=5)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.score(X_test, y_test))

##Melhor hiperparâmetros